# Exploratory Data Analysis - DSCI 522 Group 8

## Authors

* Linhan Cai

GitHub link to group repository: https://github.com/UBC-MDS/online_news_popularity


## Introduction
A project assessing factors associated with online news popularity in accordance with DSCI 522 (Data Science Workflows) for the Master of Data Science Program at the University of British Columbia.

Initial exploratory data analysis of OnlineNewsPopularity dataset. Using this Oniline News popularity dataset, our aim is to answer the question: Figure out the missing data, and find out what analysis we can do with  the dataset.

Creators: Kelwin Fernandes (kafc ‘@’ inesctec.pt, kelwinfc ’@’ gmail.com),
                 Pedro Vinagre (pedro.vinagre.sousa ’@’ gmail.com) and
                 Pedro Sernadela
Donor: Kelwin Fernandes (kafc ’@’ inesctec.pt, kelwinfc '@' gmail.com)
Date: May, 2015

Past Usage:
       K. Fernandes, P. Vinagre and P. Cortez. A Proactive Intelligent Decision
       Support System for Predicting the Popularity of Online News. Proceedings
       of the 17th EPIA 2015 - Portuguese Conference on Artificial Intelligence,
       September, Coimbra, Portugal.

       -- Results: 
          -- Binary classification as popular vs unpopular using a decision
             threshold of 1400 social interactions.
          -- Experiments with different models: Random Forest (best model),
             Adaboost, SVM, KNN and Naïve Bayes.
          -- Recorded 67% of accuracy and 0.73 of AUC.
  

## EDA:

### Import Packages

In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import altair as alt
import matplotlib.pyplot as plt
import mglearn
from imageio import imread
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Packages necessary for data splitting
from sklearn.model_selection import train_test_split

### Import the Dataset

In [3]:
ONP_csv = pd.read_csv("../data/raw/OnlineNewsPopularity/OnlineNewsPopularity.csv")

### Explore Dataset Features

In [4]:
ONP_csv.head(10)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505
5,http://mashable.com/2013/01/07/beewi-smart-toys/,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,0.0,...,0.136364,0.6,-0.195000,-0.400,-0.100000,0.642857,0.214286,0.142857,0.214286,855
6,http://mashable.com/2013/01/07/bodymedia-armba...,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,20.0,...,0.100000,1.0,-0.224479,-0.500,-0.050000,0.000000,0.000000,0.500000,0.000000,556
7,http://mashable.com/2013/01/07/canon-poweshot-n/,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,20.0,...,0.100000,1.0,-0.242778,-0.500,-0.050000,1.000000,0.500000,0.500000,0.500000,891
8,http://mashable.com/2013/01/07/car-of-the-futu...,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,0.0,...,0.400000,0.8,-0.125000,-0.125,-0.125000,0.125000,0.000000,0.375000,0.000000,3600
9,http://mashable.com/2013/01/07/chuck-hagel-web...,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,1.0,...,0.100000,0.5,-0.238095,-0.500,-0.100000,0.000000,0.000000,0.500000,0.000000,710


In [5]:
ONP_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39644 entries, 0 to 39643
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   url                             39644 non-null  object 
 1    timedelta                      39644 non-null  float64
 2    n_tokens_title                 39644 non-null  float64
 3    n_tokens_content               39644 non-null  float64
 4    n_unique_tokens                39644 non-null  float64
 5    n_non_stop_words               39644 non-null  float64
 6    n_non_stop_unique_tokens       39644 non-null  float64
 7    num_hrefs                      39644 non-null  float64
 8    num_self_hrefs                 39644 non-null  float64
 9    num_imgs                       39644 non-null  float64
 10   num_videos                     39644 non-null  float64
 11   average_token_length           39644 non-null  float64
 12   num_keywords                   

In [6]:
ONP_csv.describe(include = 'all')

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
count,39644,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,...,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
unique,39644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,http://mashable.com/2013/01/07/amazon-instant-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,354.530471,10.398749,546.514731,0.548216,0.996469,0.689175,10.883690,3.293638,4.544143,...,0.095446,0.756728,-0.259524,-0.521944,-0.107500,0.282353,0.071425,0.341843,0.156064,3395.380184
std,NaN,214.163767,2.114037,471.107508,3.520708,5.231231,3.264816,11.332017,3.855141,8.309434,...,0.071315,0.247786,0.127726,0.290290,0.095373,0.324247,0.265450,0.188791,0.226294,11626.950749
min,NaN,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,1.000000
25%,NaN,164.000000,9.000000,246.000000,0.470870,1.000000,0.625739,4.000000,1.000000,1.000000,...,0.050000,0.600000,-0.328383,-0.700000,-0.125000,0.000000,0.000000,0.166667,0.000000,946.000000
50%,NaN,339.000000,10.000000,409.000000,0.539226,1.000000,0.690476,8.000000,3.000000,1.000000,...,0.100000,0.800000,-0.253333,-0.500000,-0.100000,0.150000,0.000000,0.500000,0.000000,1400.000000
75%,NaN,542.000000,12.000000,716.000000,0.608696,1.000000,0.754630,14.000000,4.000000,4.000000,...,0.100000,1.000000,-0.186905,-0.300000,-0.050000,0.500000,0.150000,0.500000,0.250000,2800.000000


Initial data exploration shows that we have different types of features (binary, numerical...), and some of these features also have missing values. The dataset is large and representative.

### Pandas Profiling
Use Pandas Profiling to create a report about the dataset, including information about each feature, and possible correlations between features. The report is outputted as an interactable html file named `pandas_profiling.html`.

In [9]:
'''
profile = ProfileReport(ONP_csv, title='ONP')
#profile.to_notebook_iframe() # create pandas profiling report in notebook
profile.to_file("pandas_profiling.html") # create pandas profiling report in an html file
'''

'\nprofile = ProfileReport(ONP_csv, title=\'ONP\')\n#profile.to_notebook_iframe() # create pandas profiling report in notebook\nprofile.to_file("pandas_profiling.html") # create pandas profiling report in an html file\n'

It throw the error MemoryError: Unable to allocate 30.0 TiB for an array with shape (4117972210849,) and data type float64. Seems like we may need to truncate our data set.


In [57]:
from pandas_profiling import ProfileReport

profile = ProfileReport(ONP_csv, title='Pandas Profiling Report', minimal=True)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### Split the data into train and test set

In [46]:
ONP_csv_ = ONP_csv.dropna()
train_df, test_df = train_test_split(ONP_csv, test_size=0.3, random_state=42)

In [47]:
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
6286,http://mashable.com/2013/05/02/robot-fly/,616.0,9.0,744.0,0.497970,1.0,0.689655,9.0,0.0,0.0,...,0.1,0.8,-0.249621,-0.600000,-0.083333,0.900000,0.800000,0.400000,0.800000,3800
36285,http://mashable.com/2014/11/05/magic-tricks-90...,63.0,12.0,623.0,0.503268,1.0,0.675603,5.0,4.0,9.0,...,0.1,1.0,-0.235185,-0.700000,-0.100000,0.100000,0.200000,0.400000,0.200000,1500
12083,http://mashable.com/2013/08/26/nebraska-footba...,500.0,14.0,493.0,0.582485,1.0,0.777778,9.0,0.0,1.0,...,0.1,1.0,-0.312500,-0.500000,-0.125000,0.727273,0.318182,0.227273,0.318182,974
7859,http://mashable.com/2013/06/04/nba-finals-anim...,583.0,10.0,126.0,0.821138,1.0,0.892857,4.0,3.0,0.0,...,0.5,0.5,-0.433333,-0.433333,-0.433333,0.650000,0.350000,0.150000,0.350000,2000
12702,http://mashable.com/2013/09/09/government-data...,486.0,11.0,238.0,0.608696,1.0,0.773050,5.0,3.0,1.0,...,0.1,0.8,-0.316667,-0.500000,-0.050000,0.066667,0.000000,0.433333,0.000000,1600


In [48]:
train_df.describe()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
count,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,...,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000,27750.000000
mean,354.351856,10.403856,550.164613,0.555211,1.007532,0.696010,10.860505,3.296216,4.550414,1.254991,...,0.094863,0.757122,-0.259783,-0.522282,-0.107760,0.282392,0.070954,0.342277,0.156543,3373.312541
std,213.593922,2.124787,470.523540,4.207166,6.251632,3.900970,11.221066,3.838467,8.320699,4.178646,...,0.070337,0.248100,0.128101,0.289994,0.096598,0.324514,0.266772,0.188409,0.227366,10922.210179
min,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,1.000000
25%,166.000000,9.000000,247.000000,0.470296,1.000000,0.625726,4.000000,1.000000,1.000000,0.000000,...,0.050000,0.600000,-0.327972,-0.700000,-0.125000,0.000000,0.000000,0.166667,0.000000,947.000000
50%,339.000000,10.000000,414.000000,0.538613,1.000000,0.690583,8.000000,3.000000,1.000000,0.000000,...,0.100000,0.800000,-0.253568,-0.500000,-0.100000,0.141667,0.000000,0.500000,0.000000,1400.000000
75%,541.000000,12.000000,721.000000,0.608318,1.000000,0.754386,14.000000,4.000000,4.000000,1.000000,...,0.100000,1.000000,-0.187500,-0.300000,-0.050000,0.500000,0.145739,0.500000,0.250000,2800.000000
max,731.000000,23.000000,7185.000000,701.000000,1042.000000,650.000000,304.000000,116.000000,128.000000,91.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000,690400.000000


In [49]:
train_df.info

<bound method DataFrame.info of                                                      url   timedelta  \
6286           http://mashable.com/2013/05/02/robot-fly/       616.0   
36285  http://mashable.com/2014/11/05/magic-tricks-90...        63.0   
12083  http://mashable.com/2013/08/26/nebraska-footba...       500.0   
7859   http://mashable.com/2013/06/04/nba-finals-anim...       583.0   
12702  http://mashable.com/2013/09/09/government-data...       486.0   
...                                                  ...         ...   
6265              http://mashable.com/2013/05/02/iflask/       616.0   
11284  http://mashable.com/2013/08/10/internet-securi...       516.0   
38158  http://mashable.com/2014/12/03/indiegogo-insur...        35.0   
860    http://mashable.com/2013/01/22/obama-sings-sex...       716.0   
15795  http://mashable.com/2013/11/07/space-burial-co...       427.0   

        n_tokens_title   n_tokens_content   n_unique_tokens  \
6286               9.0              744.

### Initial EDA

In [33]:
y_train = train_df[' shares']
X_train = train_df.drop(columns=[' shares'])
X_test = test_df.drop(columns=[' shares'])
y_test = test_df[' shares']

In [34]:
numeric_features = [
' timedelta', ' n_tokens_title', ' n_tokens_content', ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens', ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos', ' average_token_length', ' num_keywords',
' kw_min_min', ' kw_max_min', ' kw_avg_min', ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg', ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares', ' self_reference_max_shares', ' self_reference_avg_sharess',
' global_subjectivity', ' global_sentiment_polarity', ' global_rate_positive_words', ' global_rate_negative_words', ' rate_positive_words', ' rate_negative_words', ' avg_positive_polarity', ' min_positive_polarity', ' max_positive_polarity', ' avg_negative_polarity', 
    ' min_negative_polarity', ' max_negative_polarity', ' title_subjectivity', ' title_sentiment_polarity', ' abs_title_subjectivity', ' abs_title_sentiment_polarity',' shares'
]

In [35]:
looking_columns = X_train.select_dtypes(include=np.number).columns.tolist()
print(looking_columns)

[' timedelta', ' n_tokens_title', ' n_tokens_content', ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens', ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos', ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle', ' data_channel_is_entertainment', ' data_channel_is_bus', ' data_channel_is_socmed', ' data_channel_is_tech', ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min', ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg', ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares', ' self_reference_max_shares', ' self_reference_avg_sharess', ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02', ' LDA_03', ' LDA_04', ' global_subjectivity', ' global_sentiment_polarity', ' global_rate_positive_words', ' global_rate_negative_words', ' rate_positive_words', ' rate_n

### Determining Which Numerical Features Are Important for Shares

In [36]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [40]:
df_cor = ONP_csv[numeric_features]

corrmat = df_cor.corr(method='pearson')
f, ax = plt.subplots(figsize=(60, 60))

sns.heatmap(corrmat, vmax=0.5, square=True, cmap=plt.cm.Blues)
plt.title("Correlation map", fontsize=16)
plt.show()

The correlation heat map for our numeric features shows that the shares has not very strong correlations with the other numeric features. all the correlation is between 0.1-0.3.

### Identify numeric, categorical, binary, and other features

In [45]:
numeric_features = [
' timedelta', ' n_tokens_title', ' n_tokens_content', ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens', ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos', ' average_token_length', ' num_keywords',
' kw_min_min', ' kw_max_min', ' kw_avg_min', ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg', ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares', ' self_reference_max_shares', ' self_reference_avg_sharess',
' global_subjectivity', ' global_sentiment_polarity', ' global_rate_positive_words', ' global_rate_negative_words', ' rate_positive_words', ' rate_negative_words', ' avg_positive_polarity', ' min_positive_polarity', ' max_positive_polarity', ' avg_negative_polarity', 
    ' LDA_00', ' LDA_01', ' LDA_02', ' LDA_03', ' LDA_04',' min_negative_polarity', ' max_negative_polarity', ' title_subjectivity', ' title_sentiment_polarity', ' abs_title_subjectivity', ' abs_title_sentiment_polarity',' shares'
]
binary_features=[' data_channel_is_lifestyle', ' data_channel_is_entertainment', ' data_channel_is_bus', ' data_channel_is_socmed', ' data_channel_is_tech', ' data_channel_is_world',
                ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday', ' is_weekend'
                ]
target = ' shares'

Just the experimental class for the features. need do more to explore.

### Violin Plots of binary features

In [54]:
alt.data_transformers.enable('csv')

DataTransformerRegistry.enable('csv')

In [46]:
boxplot = (alt.Chart(train_df).mark_boxplot().encode(
    x=alt.X(alt.repeat(), type='nominal'),
    y=' shares'
).properties(height=200, width=100
).repeat([' data_channel_is_lifestyle', ' data_channel_is_entertainment', ' data_channel_is_bus', ' data_channel_is_socmed', ' data_channel_is_tech', ' data_channel_is_world',
                ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday', ' is_weekend']
))
boxplot

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

alt.RepeatChart(...)

### References

Online News Popularity. (2015). UCI Machine Learning Repository. Available at: https://archive-beta.ics.uci.edu/ml/datasets/online+news+popularity.